# Assignment 4

In [1]:
import networkx as nx
print(nx.__version__)

2.4


In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [15]:
# P1_Graphs = []
# for i in range(5):
#     nodes, edges = nx.read_gpickle('A4_graphs_%d'%i)
#     G = nx.Graph()
#     G.add_nodes_from(nodes)
#     G.add_edges_from(edges)
#     P1_Graphs.append(G)
# P1_Graphs

In [4]:
# node_list = [] 
# label_list = []
# for i in range(len(P1_Graphs)):
#     node_list += list(P1_Graphs[i].nodes())
#     label_list += [i]*len(list(P1_Graphs[i].nodes()))
# nodes_df = pd.DataFrame()
# nodes_df['node'] = node_list
# nodes_df['label'] = label_list
# nodes_df.shape

# nodes_df.to_csv('A_nodes.csv', index=False)


# # list(P1_Graphs[i].edges())
# from_list = [] 
# to_list = []

# label_list = []
# for i in range(len(P1_Graphs)):
#     from_list += [i[0] for i in list(P1_Graphs[i].edges())]
#     to_list += [i[1] for i in list(P1_Graphs[i].edges())]
#     label_list += [i]*len(list(P1_Graphs[i].edges()))
# edges_df = pd.DataFrame()
# edges_df['from'] = from_list
# edges_df['to'] = to_list
# edges_df['label'] = label_list
# edges_df.shape

# edges_df.to_csv('A_edges.csv', index=False)

In [5]:
nodes = pd.read_csv('A_nodes.csv')
edges = pd.read_csv('A_edges.csv')

P1_Graphs = []
for i in range(5):
    G = nx.Graph()
    G.add_nodes_from(list(nodes[nodes.label==i].node))
    from_list = list(edges[edges.label==i]['from'])
    to_list = list(edges[edges.label==i]['to'])
    G.add_edges_from(list((from_list[i], to_list[i]) for i in range(len(from_list))))
    P1_Graphs.append(G)
P1_Graphs

In [6]:
def degree_distribution(G):
    degrees = G.degree()
    degree_values = sorted(set(degrees.values()))
    histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes( G)) for i in degree_values]
    return histogram

def graph_identification():
    methods = []
    for G in P1_Graphs:
        clustering = nx.average_clustering(G)
        shortest_path = nx.average_shortest_path_length(G)
#         degree_hist = degree_distribution(G)  # original
        degree_hist = nx.degree_histogram(G)
        if len(degree_hist)>10:
            methods.append('PA')
        elif clustering < 0.1:
            methods.append('SW_H')
        else:
            methods.append('SW_L')
    return methods
graph_identification()

['PA', 'PA', 'SW_L', 'PA', 'PA']

---

## Part 2 - Company Emails

For the second part of this assignment you will be working with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [7]:
nodes, edges = nx.read_gpickle('email_prediction_v2.pickle')
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

In [8]:
# email_node_df = pd.DataFrame()
# email_node_df['node'] = list(G.nodes())
# attr1_dict = nx.get_node_attributes(G,'Department')
# attr2_dict = nx.get_node_attributes(G,'ManagementSalary')
# email_node_df['Department'] = email_node_df['node'].apply(lambda x: attr1_dict[x])
# email_node_df['ManagementSalary'] = email_node_df['node'].apply(lambda x: attr2_dict[x])
# email_node_df.to_csv('email_prediction_nodes.csv')

# email_edge_df = pd.DataFrame()
# edge_list = list(G.edges())
# email_edge_df['from'] = [i[0] for i in edge_list]
# email_edge_df['to'] = [i[1] for i in edge_list]
# email_edge_df.to_csv('email_prediction_edges.csv')

In [9]:
email_node_df = pd.read_csv('email_prediction_nodes.csv')
email_edge_df = pd.read_csv('email_prediction_edges.csv')

nodes = list((email_node_df.node[i], {'Department': email_node_df.Department[i], 'ManagementSalary': email_node_df.ManagementSalary[i]}) for i in range(email_node_df.shape[0]))

from_list = list(email_edge_df['from'])
to_list = list(email_edge_df['to'])
G.add_edges_from(list((from_list[i], to_list[i]) for i in range(len(from_list))))

G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [10]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
def salary_predictions():
    def is_management(node):
        managementSalary = node[1]['ManagementSalary']
        if managementSalary == 0:
            return 0
        elif managementSalary == 1:
            return 1
        else:
            return None

    df = pd.DataFrame(index=G.nodes())
    df['clustering'] = pd.Series(nx.clustering(G))
#     df['degree'] = pd.Series(G.degree())  # original
    df['degree'] = pd.Series(dict(G.degree()))
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G))
#     df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True)) # original
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['pr'] = pd.Series(nx.pagerank(G))
    df['is_management'] = pd.Series([is_management(node) for node in G.nodes(data=True)])
    df_train = df[~pd.isnull(df['is_management'])]
    df_test = df[pd.isnull(df['is_management'])]
    features = ['clustering', 'degree', 'degree_centrality', 'closeness', 'betweeness', 'pr']
    X_train = df_train[features]
    Y_train = df_train['is_management']
    X_test = df_test[features]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5,
                       random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(X_train_scaled, Y_train)
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    return pd.Series(test_proba,X_test.index)
# prediction = salary_predictions()

In [11]:
salary_predictions()

1       0.128439
2       0.586261
5       0.979859
8       0.133702
14      0.304625
          ...   
992     0.003956
994     0.003077
996     0.002429
1000    0.046944
1001    0.101407
Length: 252, dtype: float64

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [12]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [13]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
def new_connections_predictions():
    for node in G.nodes():
        G.nodes[node]['community'] = G.nodes[node]['Department']
    preferential_attachment = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index=[(x[0], x[1]) for x in preferential_attachment])
    df['preferential_attachment'] = [x[2] for x in preferential_attachment]
    cn_soundarajan_hopcroft = list(nx.cn_soundarajan_hopcroft(G))
    df_cn_soundarajan_hopcroft = pd.DataFrame(index=[(x[0], x[1]) for x in cn_soundarajan_hopcroft])
    df_cn_soundarajan_hopcroft['cn_soundarajan_hopcroft'] = [x[2] for x in cn_soundarajan_hopcroft]
    df = df.join(df_cn_soundarajan_hopcroft,how='outer')
    df['cn_soundarajan_hopcroft'] = df['cn_soundarajan_hopcroft'].fillna(value=0)
    df['resource_allocation_index'] = [x[2] for x in list(nx.resource_allocation_index(G))]
    df['jaccard_coefficient'] = [x[2] for x in list(nx.jaccard_coefficient(G))]
    df = future_connections.join(df,how='outer')
    df_train = df[~pd.isnull(df['Future Connection'])]
    df_test = df[pd.isnull(df['Future Connection'])]
    features = ['cn_soundarajan_hopcroft', 'preferential_attachment', 'resource_allocation_index', 'jaccard_coefficient']
    X_train = df_train[features]
    Y_train = df_train['Future Connection']
    X_test = df_test[features]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5,
                       random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(X_train_scaled, Y_train)
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    predictions = pd.Series(test_proba,X_test.index)
    target = future_connections[pd.isnull(future_connections['Future Connection'])]
    target['prob'] = [predictions[x] for x in target.index]
    return target['prob']
# new_connections_predictions()

In [14]:
new_connections_predictions()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(107, 348)    0.024102
(542, 751)    0.010265
(20, 426)     0.543569
(50, 989)     0.010248
(942, 986)    0.010230
                ...   
(165, 923)    0.010437
(673, 755)    0.010229
(939, 940)    0.010230
(555, 905)    0.010275
(75, 101)     0.020768
Name: prob, Length: 122112, dtype: float64